In [1]:
import requests
import pandas as pd
import talib
import numpy as np
import time

# Replace with your OANDA API key and account ID
api_key = '0f11fb98df9be27e57dcf0a797c8690d-735e79da881ec985e1f22617060bdbd9'
account_id = '101-011-24509333-001'

# OANDA API endpoints
base_url = 'https://api-fxpractice.oanda.com/v3'
instrument = 'EUR_USD'
granularity = 'H1'  # Hourly timeframe

# SSL indicator parameters
fast_length = 10
slow_length = 20
signal_length = 5

# Function to fetch historical candlestick data from OANDA
def fetch_candles(instrument, granularity, count=100):
    url = f"{base_url}/instruments/{instrument}/candles"
    params = {
        'granularity': granularity,
        'count': count,
    }
    headers = {
        'Authorization': f'Bearer {api_key}',
    }
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    
    candles = data['candles']
    df = pd.DataFrame(candles)
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    
    return df

# Function to calculate SSL indicator
def calculate_ssl(data):
    high = data['high'].values.astype(float)
    low = data['low'].values.astype(float)
    close = data['close'].values.astype(float)

    ssl_values = talib.SSL(high, low, close, fast_length=fast_length, slow_length=slow_length, signal_length=signal_length)
    data['SSL_Up'] = np.where(ssl_values > 0, 1, 0)
    data['SSL_Down'] = np.where(ssl_values < 0, 1, 0)

    return data

while True:
    try:
        # Fetch historical candlestick data
        historical_data = fetch_candles(instrument, granularity, count=fast_length + signal_length - 1)

        # Calculate SSL indicator
        data_with_ssl = calculate_ssl(historical_data)

        # Print the latest SSL values
        latest_ssl_values = data_with_ssl[['SSL_Up', 'SSL_Down']].tail(1)
        print("Latest SSL Values:")
        print(latest_ssl_values)

    except Exception as e:
        print(f"An error occurred: {e}")

    time.sleep(60)  # Sleep for 60 seconds before checking again


ModuleNotFoundError: No module named 'talib'

In [2]:
!pip install talib

ERROR: Could not find a version that satisfies the requirement talib (from versions: none)
ERROR: No matching distribution found for talib
